In [14]:
import ROOT
from root_numpy import root2array, tree2array, root2rec, testdata
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize # MINIMIZATION
import copy

## Prepare data structure and functions for single tracking

### Line

In [2]:
class Line:
    def __init__(self, x=0, y=0, z=0, dx=0, dy=0, dz=0, point=[], direction=[], params=[], det_id=0):
        if len(params) == 6:
            self.x,  self.y,  self.z  = params[:3] 
            self.dx, self.dy, self.dz = params[3:]
        elif len(point) == 3 and len(direction) == 3:
            self.x,  self.y,  self.z  = point
            self.dx, self.dy, self.dz = direction
        else:
            self.x,  self.y,  self.z  =  x,  y,  z
            self.dx, self.dy, self.dz = dx, dy, dz
        
        # just to make this line a detector Line
        self.det_id = int(det_id)

    def __repr__(self):
        return "Line: det = {}\t p = ({:.3f}, {:.3f}, {:.3f})\t v = [{:.3f}, {:.3f}, {:.3f}]".format(
            self.det_id, self.x, self.y, self.z, self.dx, self.dy, self.dz
        )
            
    def distance(self, other):
        n_vector = np.cross(self.line_vector(), other.line_vector())
        s_o_vector = np.array([self.x - other.x, self.y - other.y, self.z - other.z])
        return np.linalg.norm( np.dot(n_vector, s_o_vector) ) / np.linalg.norm(n_vector)

    def line_vector(self):
        return np.array([self.dx, self.dy, self.dz])

### Open root file

In [3]:
geometry_root = root2array('det_geometry.root', 'T')
reco_flat = root2array('reco_10k_flat.root', 'T')

### Prepare dataframes

In [4]:
geom_df = pd.DataFrame(geometry_root) 
df_all  = pd.DataFrame(reco_flat)

single_df = df_all.loc[(df_all['uLineSize'] == 1) & (df_all['vLineSize'] == 1)]     # OLNY RPS WITH SINGLE TRACKING
single_groups_df = single_df[['eventID', 'groupID', 'rpID']]\
    .drop_duplicates()\
    .groupby(['eventID', 'groupID'])\
    .size()\
    .reset_index(name='counts')
single_groups_df = single_groups_df.loc[(single_groups_df['counts'] == 3)]          # ONLY HITS WHERE ALL RPS 
                                                                                    # FROM GROUP WAS HIT

print("all tracks shape: {}".format(df_all.shape))
print("single tracking shape: {}".format(single_df.shape))
print("single_groups shape: {}".format(single_groups_df.shape))

all tracks shape: (569625, 10)
single tracking shape: (502443, 10)
single_groups shape: (15320, 3)


### Apply fiducials to detector: Move center by [dx, dy] defined in Geometry/TotemRPData/data/RP_Hybrid.xml

In [5]:
class Direction:
    '''
    Wektor jednostkowy
    '''
    def __init__(self, dx=1.0, dy=0.0):
        self.dx = dx
        self.dy = dy
    
    def __repr__(self):
        return "dx = {}\tdy={}".format(self.dx, self.dy)

    def get_perpendicular(self):
        return Direction(self.dy, -self.dx)
        
    
class Vector2D:
    def __init__(self, x=0.0, y=0.0, start=None, end=None):
        if start is None or end is None:
            self.x = x
            self.y = y
        else:
            self.x = end.x - start.x
            self.y = end.y - start.y
                 
    def __repr__(self):
        return "x = {}\ty = {}".format(self.x, self.y)
    

class Point2D:
    def __init__(self, x=0.0, y=0.0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return "x = {}\ty = {}".format(self.x, self.y)
    
    def move(self, vector):
        self.x += vector.x
        self.y += vector.y
    
    def rotate_around_point(self, other, direction):
        s = direction.dy
        c = direction.dx
        
        new_x = c * (self.x - other.x) - s * (self.y - other.y) + other.x
        new_y = s * (self.x - other.x) + c * (self.y - other.y) + other.y
    
        self.x = new_x
        self.y = new_y

        
class RPSilicon:
    
    RP_Det_Fid_Top_a   = 0.301  # [mm]
    RP_Det_Fid_Top_b   = 1.0275 # [mm]
    RP_Det_Fid_Left_a  = 0.301  # [mm]
    RP_Det_Fid_Left_c  = 0.2985 # [mm]
    RP_Det_Fid_Right_b = 1.0275 # [mm]
    RP_Det_Fid_Right_d = 1.0255 # [mm]

    RP_Det_dx_0 = ( RP_Det_Fid_Left_c  -  RP_Det_Fid_Right_b) / 2.0 # [mm]
    RP_Det_dy_0 = ( RP_Det_Fid_Right_d  -  RP_Det_Fid_Left_a) / 2.0
    
    def __init__(self, center, direction, siId):
        self.center = center
        self.direction = direction
        self.siId = siId
        
        fid_center = copy.deepcopy(self.center)
        fid_center.move(Vector2D(RPSilicon.RP_Det_dx_0, RPSilicon.RP_Det_dy_0))
        fid_center.rotate_around_point(self.center, self.get_rot_direction())
        
        self.fid_center = fid_center
    
    def get_rot_direction(self):
        arm = int(self.siId / 1000) 
        if   arm == 0 and self.siId % 2 == 0:  
            return self.direction
        elif arm == 0 and self.siId % 2 == 1:
            return Direction(self.direction.dy, -self.direction.dx)
        elif arm == 1 and self.siId % 2 == 0:
            return Direction(self.direction.dy, -self.direction.dx)
        else:
            return self.direction

### Create hit lines for given hits dataframe

In [6]:
DET_Z_TRANSLATION = 0 # SET WITHIN EVENT PROCESSING


def get_hit_lines(single_hits_df, geom_df):
    '''
        reco_0_flat.root:     recoID  eventID  armID  groupID  rpID  uLineSize  vLineSize  siliconID   position  sigma_
        det_geometry.root:    detId   x        y      z        dx    dy
    '''
    
    hit_lines = []

    for h_index, h_row in single_hits_df.iterrows():
        
        h_position = h_row['position']
        det_ID = 10 * h_row['rpID'] + h_row['siliconID']        
        det_info = geom_df.loc[(geom_df['detId'] == det_ID)].iloc[0]
    
        rp_silicon = RPSilicon(Point2D(det_info['x'], det_info['y']), Direction(dx=det_info['dx'], dy=det_info['dy']), det_ID)
        
        x = rp_silicon.fid_center.x + h_position * det_info['dx']    # was: det_info['x'], but now: fiducial included [mm]
        y = rp_silicon.fid_center.y + h_position * det_info['dy']    # was: det_info['y'], but now: fiducial included [mm] 
        
        z = det_info['z'] * 1000 + DET_Z_TRANSLATION     # z [mm] with translation to operate on smaller numbers
        dx = - det_info['dy']  
        dy = det_info['dx'] 
        dz = 0
        
        new_line = Line(x, y, z, dx, dy, dz, det_id=det_ID)
        hit_lines.append(new_line)
        
    return hit_lines

### Printing fitting solution

In [7]:
class Hit:
    def __init__(self, x=0, y=0, z=0):
        self.x = x
        self.y = y
        self.z = z


def print_pretty_solution(solution, geom_df):
    print("FULL SOLUTION: \n{}".format(solution))
    print("Sum of distances: {}".format(solution.fun))
    print("Solution: [\n\tx = {}\n\ty = {} \n\tz = {} \n\tdx = {} \n\tdy = {} \n\tdz = {} ]".format(
        solution.x[0],
        solution.x[1],
        solution.x[2], # - DET_Z_TRANSLATION,
        solution.x[3],
        solution.x[4],
        solution.x[5])
    )
    print("]")
    print("\nDistances from detector lines and hit position")
    print("ID\tDistance from hit line [mm]\t x [mm]\t\ty [mm]\t\tz [m]")

    plane_hit = Hit()

    for line in LINE_SET:
        sol_x, sol_y, sol_z = solution.x[0], solution.x[1], solution.x[2]
        sol_dx, sol_dy, sol_dz = solution.x[3], solution.x[4], solution.x[5]
        
        det_info = geom_df.loc[(geom_df['detId'] == line.det_id)].iloc[0]
        det_z = det_info['z']
        k = (det_z - (sol_z - DET_Z_TRANSLATION)) / sol_dz
        
        plane_hit.x = sol_x + k * sol_dx
        plane_hit.y = sol_y + k * sol_dy
        plane_hit.z = det_z

        print("{}\t{:.5f}\t\t\t\t({:.5f}\t{:.5f}\t{})".format(line.det_id,
                                                              Line(params=solution.x).distance(line),
                                                              plane_hit.x,
                                                              plane_hit.y,
                                                              plane_hit.z))
    print("\n")

## Minimization for each rp group where is single-tracking 

### Chi 2

In [11]:
# np.sqrt(2)

1.4142135623730951

In [12]:
'''
    Chi^2 = Sum ( (O_i - E_i) / sigma_i ) ^ 2
    
    E_i - Expected value (We want distance to hit line = 0)
    O_i - Observed value (Actual distance to hit line)
    
    sigma_i =
        a) sigma from DataFormats/TotemRPDataTypes/interface/RPRecoHit.h
        b) sigma as a distances between two strips - 0.0659
        c) sqrt( Sum[(x_i - u)^2] / N )
            x_i - i-th distance from hit line
            u   - average distance from hit line
'''

# REQUIRED BY MY CHI2 FUNCTION
LINE_SET = []

# SIGMA = 0.0191  # a)
SIGMA = 0.0659 / np.sqrt(2)  # b)

def chi2(params):
    line = Line(params=params)                                                        # CREATING FITTED LINE FROM PARAMS
    return np.sum([np.power( line.distance(other) / SIGMA, 2) for other in LINE_SET]) # SUM OF DISTANCES

# c)
def get_sigma(params):
    line = Line(params=params)
    distances = np.array([line.distance(other) for other in LINE_SET])
    
    u = np.average(distances)
    N = len(distances)
    u_diff = np.array([dist - u for dist in distances])
    
    return np.sqrt( np.sum(np.power(u_diff, 2.0)) / len(distances) )

def chi2_wiki(params):
    line = Line(params=params)
    sigma = get_sigma(params)
    return np.sum([np.power( line.distance(other) / sigma, 2) for other in LINE_SET])

### Minimization

In [20]:
# OBJECTIVE FUNCTION
def objective(params):
    line = Line(params=params)                                  # CREATING FITTED LINE FROM PARAMS
    return np.sum([line.distance(other) for other in LINE_SET]) # SUM OF DISTANCES

# SEED FOR MINIMIZING
def get_x0(first_hit_info, geom_df):    
    det_id   = first_hit_info['rpID'] * 10 + first_hit_info['siliconID'] 
    det_info = geom_df.loc[(geom_df['detId'] == det_id)].iloc[0]
    
    det_x = 0.0 # det_info['x']
    det_y = 0.0 # det_info['y']
    det_z = det_info['z'] * 1000 + DET_Z_TRANSLATION 
    
    return [det_x, det_y, det_z, 0.1, 0.1, np.sign(det_z)]

def get_constraints():
    def constraint1(params):
        return 1.0 - np.sum([dir**2 for dir in params[3:]]) # dx^2 + dy^2 + dz^2 = 1

    con1 = {'type': 'eq', 'fun':constraint1}
    return [con1]

# BOUNDS FOR PARAMETERS
def get_bounds():
    b_x   = (   -50.0,    50.0)
    b_y   = (   -50.0,    50.0)
    b_z   = (-20000.0, 20000.0)
    b_dir = (    -1.0,     1.0)
    return (b_x, b_y, b_z, b_dir, b_dir, b_dir)


def is_good(solution):
    return solution.fun < 100.0

def get_solution():
    # SEED SOLUTION - CENTER OF FIRST PLANE [x, y, z] AND DIRECTION ALONG THE BEAM [0, 0, [+|-] 1] 
    x0 = get_x0(single_hits_df.iloc[0], geom_df)    
    
    # RUNNING OPTIMIZING METHODS TO GET SOLUTION
    OPTIMIZING_METHODS = ['SLSQP', 'Nelder-Mead', 'Powell']
    
    for method in OPTIMIZING_METHODS:
        solution = minimize(objective, x0, method=method, constraints=get_constraints(), bounds=get_bounds())
        
        if solution.success and is_good(solution):
            break
            
    return (solution, method) 

### Result Dataframe initialization

In [48]:
def get_column_names():
    return ['Row', 'Event', 'Group', 'Method', 'DistSum', 'MDH', 'Success', 'Chi', 'ChiN', 'x', 'y', 'z', 'dx', 'dy', 'dz']

def get_empty_row_data():
    result = {}
    column_names = get_column_names()
    
    for column in column_names:
        result[column] = []
    
    return result

def get_row_data_dict(row_data):
    result = {}
    columns = get_column_names()
    
    for i, (key, val) in enumerate(zip(columns, row_data)):
        result[key] = [val]
    
    return result
    
def get_row_df(row_data):
    row_data_dict = get_row_data_dict(row_data)
    columns = get_column_names()
    
    return pd.DataFrame(row_data_dict, columns = columns)

result_df = pd.DataFrame(get_empty_row_data(), columns = get_column_names())

### Itaration over Events and Group with single tracking

In [50]:
# HELP VARIABLES
row_number = 0
glob_max_hit_dist = 0.0
true_max_distance = 0.0
false_max_distance = 0.0


# FOR EACH EVENT AND GROUP CALCULATE LINE
for index, row in single_groups_df.iterrows():

    if row_number > 20:
        break
    
    # GETTING HITS FOR SPECIFIED GROUP AND EVENT
    eventID, groupID = row['eventID'], row['groupID']
    single_hits_df = single_df.loc[(single_df['eventID'] == eventID) & (single_df['groupID'] == groupID)]
    
    # APPLYING Z TRANSLATION --> WE DO NOT WANT SUCH BIG NUMBERS (|Z| ~ 210 000 [mm])
    if groupID > 3:
        DET_Z_TRANSLATION = -210000
    else:
        DET_Z_TRANSLATION = 210000

    # GET LINES REPRESENTING HITS IN RPS
    LINE_SET = get_hit_lines(single_hits_df, geom_df)  
    
    # GET FITTED LINE AND METHOD USED FOR FITTING
    solution, method = get_solution()
    
    # UPDATE STATISTICS
    if solution.success == True:
        true_max_distance = np.maximum(true_max_distance, solution.fun)
    else:
        false_max_distance = np.maximum(false_max_distance, solution.fun)
                
    
    distances = [ Line(params=solution.x).distance(line) for line in LINE_SET]
    distance_sum = np.sum(distances)
    max_hit_dist = np.amax(distances)
    glob_max_hit_dist = np.maximum(glob_max_hit_dist, max_hit_dist)
    
    chi = chi2(solution.x)
    
    
    print("Row {} Event: {} Group: {} Method: {}  DistSum: {:.5f}  MDH: {:.5}  Success: {}  Chi: {:.5f}\tChiN: {:.5f}"
      .format(row_number, eventID, groupID, method, solution.fun, max_hit_dist, 
              solution.success, chi, chi / (len(LINE_SET) - 6)))

    
    # DATAFRAME FOR STORING RESULT
    x, y, z, dx, dy, dz = solution.x
    row_df = get_row_df([row_number, eventID, groupID, method, solution.fun, max_hit_dist, solution.success, 
                         chi, chi / (len(LINE_SET) - 6), x, y, z, dx, dy, dz])
    result_df = pd.concat([result_df, row_df])
    
    row_number = row_number + 1
    

print("THE END")

result_df

Row 0 Event: 2 Group: 2 Method: Nelder-Mead  DistSum: 0.42049  MDH: 0.044624  Success: True  Chi: 5.01667	ChiN: 0.21812
Row 1 Event: 2 Group: 4 Method: SLSQP  DistSum: 0.53292  MDH: 0.046286  Success: True  Chi: 7.40118	ChiN: 0.32179
Row 2 Event: 3 Group: 1 Method: SLSQP  DistSum: 0.50122  MDH: 0.067381  Success: True  Chi: 7.51384	ChiN: 0.34154
Row 3 Event: 3 Group: 2 Method: SLSQP  DistSum: 0.69759  MDH: 0.085794  Success: True  Chi: 15.64855	ChiN: 0.68037
Row 4 Event: 3 Group: 4 Method: SLSQP  DistSum: 0.49803  MDH: 0.055625  Success: True  Chi: 5.99570	ChiN: 0.24982
Row 5 Event: 4 Group: 2 Method: SLSQP  DistSum: 0.71539  MDH: 0.057979  Success: True  Chi: 12.38599	ChiN: 0.56300
Row 6 Event: 5 Group: 1 Method: Nelder-Mead  DistSum: 0.78325  MDH: 0.10546  Success: True  Chi: 18.57335	ChiN: 0.80754
Row 7 Event: 5 Group: 5 Method: SLSQP  DistSum: 0.58450  MDH: 0.055528  Success: True  Chi: 9.01701	ChiN: 0.39204
Row 8 Event: 6 Group: 1 Method: SLSQP  DistSum: 0.55677  MDH: 0.0535  Succ

,Row,Event,Group,Method,DistSum,MDH,Success,Chi,ChiN,x,y,z,dx,dy,dz
0,0.0,2.0,2.0,Nelder-Mead,0.420489,0.044624,1.0,5.016670,0.218116,-0.031184,-0.015642,4.316166e+04,0.000103,-0.002299,-1.274970e+01
0,1.0,2.0,4.0,SLSQP,0.532923,0.046286,1.0,7.401180,0.321790,-0.642334,8.870297,2.926762e+03,0.000006,0.000189,1.000000e+00
0,2.0,3.0,1.0,SLSQP,0.501218,0.067381,1.0,7.513843,0.341538,-0.300255,9.113259,-2.859422e+03,-0.000015,0.000192,-1.000000e+00
0,3.0,3.0,2.0,SLSQP,0.697590,0.085794,1.0,15.648552,0.680372,0.297102,-7.311275,-3.106754e+03,0.000013,-0.000160,-1.000000e+00
0,0.0,2.0,2.0,Nelder-Mead,0.420489,0.044624,1.0,5.016670,0.218116,-0.031184,-0.015642,4.316166e+04,0.000103,-0.002299,-1.274970e+01
0,1.0,2.0,4.0,SLSQP,0.532923,0.046286,1.0,7.401180,0.321790,-0.642334,8.870297,2.926762e+03,0.000006,0.000189,1.000000e+00
0,2.0,3.0,1.0,SLSQP,0.501218,0.067381,1.0,7.513843,0.341538,-0.300255,9.113259,-2.859422e+03,-0.000015,0.000192,-1.000000e+00
0,3.0,3.0,2.0,SLSQP,0.697590,0.085794,1.0,15.648552,0.680372,0.297102,-7.311275,-3.106754e+03,0.000013,-0.000160,-1.000000e+00
0,4.0,3.0,4.0,SLSQP,0.498031,0.055625,1.0,5.995696,0.249821,-0.605778,8.127458,2.992050e+03,-0.000001,0.000175,1.000000e+00
0,5.0,4.0,2.0,SLSQP,0.715388,0.057979,1.0,12.385986,0.562999,0.306691,-7.049259,-2.990912e+03,-0.000003,-0.000152,-1.000000e+00


In [46]:
k = [1,2,3]
a,b,c = k


a = 1, b = 2, c = 3
